In [2]:
import os
import pandas as pd
import csv

In [ ]:
# 根据“CNCB_GISAID_metadata_china_20221201-20231031.xlsx”提供的ID从GISAID下载41708条序列:"Gisaid-china.fasta",41708条
# 使用nextclade工具对其进行运算，保留nextclade.csv、nextclade.tsv、nextclade.aligned.fasta.gz结果，解压.gz文件。
# nextclade run \
#    --input-dataset data/sars-cov-2 \
#    --output-all=/home/soniali/Desktop/02_china_recom_github/0_raw_data/China_output_Gisaid \
# /home/soniali/Desktop/02_china_recom_github/0_raw_data/Gisaid-china.fasta

In [6]:
dirpath = "/home/soniali/Desktop/02_china_recom_github/0_raw_data/"
gisaid_nexttsv = dirpath+"China_output_Gisaid/nextclade.tsv"   ## 保存在14.174中/Data4/Data_public/china/output_Gisaid
gisaid_metacsv = dirpath+"CNCB_GISAID_metadata_china_20221201-20231031.xlsx"

In [7]:
df0 = pd.read_table(gisaid_nexttsv)
df = df0.loc[:,["seqName","Nextclade_pango","partiallyAliased","clade_nextstrain","clade_display","qc.overallStatus","coverage"]]
print(df.shape) # (41708,7)
df1=  df.loc[df['qc.overallStatus'] == "good"] 
print(df1.shape) #(40091, 7)
df2=  df1.loc[df['coverage'] >=0.97] 
print(df2.shape) #(40091, 7)
gisaid_qualified_ID = df2.seqName.tolist()
# qualified_ID

(41708, 7)
(40091, 7)
(40091, 7)


/tmp/ipykernel_2890823/464067568.py:1: DtypeWarning: Columns (55,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_table(gisaid_nexttsv)


In [41]:
gisaid_qualified_epi = []
for stain_name in gisaid_qualified_ID:
    if "EPI_ISL" in stain_name:
        seq_id = "EPI_"+stain_name.split("|EPI_")[1].split("|")[0]
        gisaid_qualified_epi.append(seq_id)

In [60]:
df_meta = pd.read_excel(gisaid_metacsv)
df_meta = pd.DataFrame(df_meta)
df_meta_2 = df_meta[df_meta["Accession ID"].isin(gisaid_qualified_epi)]
num = 0
chinese_qualified_epi = []
for i in df_meta_2.index:
    if str(df_meta_2.loc[i,"Sample Collection Date"]).count("-") >=2:
        if df_meta_2.loc[i,"Location"].count("/") >=1:
            chinese_qualified_epi.append(df_meta_2.loc[i,"Accession ID"])
            num+=1
num


39456

In [77]:
df_meta_qualified = df_meta_2[df_meta_2["Accession ID"].isin(chinese_qualified_epi)]
df_meta_qualified = df_meta_qualified.drop(columns=['Sequence Length'])

for i in df_meta_qualified.index:
    df_meta_qualified.loc[i,"country"] = df_meta_qualified.loc[i,"Location"].split("/")[0].strip()
    df_meta_qualified.loc[i,"province"] = df_meta_qualified.loc[i,"Location"].split("/")[1].strip()
    if len(df_meta_qualified.loc[i,"Location"].split("/")) >= 3:
        df_meta_qualified.loc[i,"city"] = df_meta_qualified.loc[i,"Location"].split("/")[2].strip()
    else:
        df_meta_qualified.loc[i,"city"] = ""
        
df_meta_qualified.head()

,Virus Strain Name,Accession ID,Lineage,Sample Collection Date,Location,country,province,city
0,hCoV-19/Henan/HBCDC-CX109/2023,EPI_ISL_18543055,EG.5.1.1,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
1,hCoV-19/Henan/HBCDC-CX108/2023,EPI_ISL_18543054,HK.2,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
2,hCoV-19/Henan/HBCDC-CX106/2023,EPI_ISL_18543053,HK.3,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
3,hCoV-19/Henan/HBCDC-CX105/2023,EPI_ISL_18543052,HK.3.2,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi
4,hCoV-19/Henan/HBCDC-CX104/2023,EPI_ISL_18543051,HK.3,2023-10-26 00:00:00,China / Henan / Hebi,China,Henan,Hebi


In [94]:
df_meta_qualified_2 = df_meta_qualified.drop(columns=['Location'])
df_meta_qualified_2['Sample Collection Date'] = pd.to_datetime(df_meta_qualified_2['Sample Collection Date'])
df_meta_qualified_2['Sample Collection Date'] = df_meta_qualified_2['Sample Collection Date'].dt.strftime('%m/%d/%Y')
df_meta_qualified_2.columns = df_meta_qualified_2.columns.str.replace(' ', '_')
chinese_qualified_ID = df_meta_qualified_2.Accession_ID.tolist()

In [93]:
len(chinese_qualified_ID)

39456

In [ ]:
from Bio import SeqIO

with open(dirpath+"Gisaid-qualified-china.fasta","a+") as f:
    with open(dirpath+"China_output_Gisaid/nextclade.aligned.fasta","r") as h: 
        for record in SeqIO.parse(h,"fasta"):
            if  {str(record.id)} - set(chinese_qualified_ID) == set():
                f.write(">"+str(record.id)+"\n")
                f.write(str(record.seq)+"\n")

In [ ]:
# 对Qualified-china.fasta进行nextclade运算，生成“0_raw_data/GISAID_nextclade”文件夹
# nextclade run \
#    --input-dataset data/sars-cov-2 \
#    --output-all=/home/soniali/Desktop/02_china_recom_github/0_raw_data/GISAID_nextclade \
# /home/soniali/Desktop/02_china_recom_github/0_raw_data/Gisaid-qualified-china.fasta

# 保留nextclade.csv、nextclade.tsv、nextclade.auspice.json、nextclade.aligned.fasta.gz结果，解压.gz文件。
# 将nextclade.auspice.json和Qualified_china_meta_auspice.tsv文件放入 https://auspice.us 网站，与全球新冠树进行比较，下载系统发育树分析结果

In [87]:
df = pd.read_csv('/home/soniali/Desktop/02_china_recom_github/0_raw_data/GISAID_nextclade/nextclade.tsv', sep='\t', usecols=['seqName', 'Nextclade_pango'])
result_dict = df.set_index('seqName')['Nextclade_pango'].to_dict()
df_meta_qualified_2.head()

In [96]:
for i in df_meta_qualified_2.index:
    temp_lin = result_dict[df_meta_qualified_2.loc[i,"Accession_ID"]]
    df_meta_qualified_2.loc[i,"Lineage"] = temp_lin

df_meta_qualified_2.head()

,Virus_Strain_Name,Accession_ID,Lineage,Sample_Collection_Date,country,province,city
0,hCoV-19/Henan/HBCDC-CX109/2023,EPI_ISL_18543055,HK.27.1,10/26/2023,China,Henan,Hebi
1,hCoV-19/Henan/HBCDC-CX108/2023,EPI_ISL_18543054,HK.2,10/26/2023,China,Henan,Hebi
2,hCoV-19/Henan/HBCDC-CX106/2023,EPI_ISL_18543053,HK.3,10/26/2023,China,Henan,Hebi
3,hCoV-19/Henan/HBCDC-CX105/2023,EPI_ISL_18543052,HK.3.2,10/26/2023,China,Henan,Hebi
4,hCoV-19/Henan/HBCDC-CX104/2023,EPI_ISL_18543051,HK.3,10/26/2023,China,Henan,Hebi


In [97]:
df_meta_qualified_2.to_csv(dirpath+"Qualified_china_meta.csv",index=False)

In [99]:
df_meta_qualified_2.shape

(39456, 7)